## Merging filtered and fit data with CCL properties

In [1]:
import pandas as pd
import numpy as np

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

import os
_FOLDER = "results/"

In [2]:
df_list = ["fit_1234_02_merged.csv", "fit_auc_02_merged.csv", "fit_no_filt_merged.csv"]

In [3]:
df = pd.read_csv(_FOLDER + "fit_1234_02_merged.csv")
df.head(2)

,CELL_LINE_NAME,COSMIC_ID,DRUG_ID,DRUGID_COSMICID,FOLD_DILUTION,MAX_CONC,fd_num_0,fd_num_1,fd_num_2,fd_num_3,...,chr9:104248247-104249501(C9orf125)_HypMET,"chr9:115875199-115875738(C9orf109, C9orf110)_HypMET",chr9:123555399-123555899(FBXW2)_HypMET,chr9:140310894-140312457(EXD3)_HypMET,chr9:21974578-21975306(CDKN2A)_HypMET,chr9:35756948-35757339(MSMP)_HypMET,chr9:35791584-35791924(NPR2)_HypMET,chr9:4984543-4985630(JAK2)_HypMET,chr9:86571047-86572027(C9orf64)_HypMET,chr9:98783216-98784364(NCRNA00092)_HypMET
0,SF539,905984,200,200_905984,2,1.024,0,0.111111,0.222222,0.333333,...,0,0,0,0,0,0,0,0,1,1
1,SF539,905984,235,235_905984,2,10.240,0,0.111111,0.222222,0.333333,...,0,0,0,0,0,0,0,0,1,1


In [4]:
df.shape[0], df["DRUGID_COSMICID"].nunique()

(1427, 1427)

### Split into train and test data with more than 10 record per drug

In [5]:
gr = df.groupby("DRUG_ID").size()
drugs = gr[gr>10].index
print("Number of drugs with more than 10 profiles:", len(drugs))        

train_ratio = 0.8
train = pd.DataFrame()
test = pd.DataFrame()
np.random.seed(123)
indexes = np.random.permutation(df.index)

for drug_id in drugs:
    df_i = df[df["DRUG_ID"]==drug_id]
    indexes = np.random.permutation(df_i.index)
    train_size = int(df_i.shape[0]*train_ratio)
    indexes_train = indexes[:train_size]
    indexes_test = indexes[train_size:]
    train = pd.concat([train, df_i.loc[indexes_train, :]])
    test = pd.concat([test, df_i.loc[indexes_test, :]])
    
train.set_index("DRUGID_COSMICID", inplace=True)
test.set_index("DRUGID_COSMICID", inplace=True)

print(train.shape, test.shape)

with open(_FOLDER +"train_10.txt", 'w') as f:
    for s in train.index:
        f.write(str(s) + '\n')
        
with open(_FOLDER +"test_10.txt", 'w') as f:
    for s in test.index:
        f.write(str(s) + '\n')

Number of drugs with more than 10 profiles: 41
(899, 1140) (255, 1140)


### Possible scenarios:
- drug-by-drug learning, i.e separate models for each fitting coefficients for each drug
- all drugs - one model for each coefficients for all drugs
- all data - one model for each response for all drugs and drug profiles